# به نام خدا

# فاز 1

In [58]:
import cv2
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm

In [59]:


def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    plates = []
    for obj in root.findall('object'):
        name = obj.find('name').text
        if name.lower() == 'vehicle plate':
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            plates.append((xmin, ymin, xmax, ymax))
    
    return plates

def extract_plates(image_path, annotation_path, output_path):
    # خواندن تصویر
    image = cv2.imread(image_path)
    if image is None:
        print(f"خطا در خواندن تصویر: {image_path}")
        return
    
    # خواندن آنوتیشن‌ها
    plates = parse_annotation(annotation_path)
    if not plates:
        print(f"هیچ پلاکی در فایل آنوتیشن یافت نشد: {annotation_path}")
        return
    
    # استخراج و ذخیره هر پلاک
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    for i, (xmin, ymin, xmax, ymax) in enumerate(plates):
        plate_img = image[ymin:ymax, xmin:xmax]
        
        # ذخیره پلاک استخراج شده
        output_file = os.path.join(output_path, f"{base_name}.png")
        cv2.imwrite(output_file, plate_img)

def process_dataset(base_dir, output_dir):
    # مسیرهای ورودی
    images_dir = os.path.join(base_dir, "Vehicle Plates", "Vehicle Plates")
    annotations_dir = os.path.join(base_dir, "Vehicle Plates annotations", "Vehicle Plates annotations")
    
    # بررسی وجود پوشه‌ها
    if not os.path.exists(images_dir):
        print(f"پوشه تصاویر یافت نشد: {images_dir}")
        return
    if not os.path.exists(annotations_dir):
        print(f"پوشه آنوتیشن‌ها یافت نشد: {annotations_dir}")
        return
    
    # لیست تمام فایل‌های تصویر
    image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    # ایجاد پوشه خروجی اگر وجود نداشته باشد
    os.makedirs(output_dir, exist_ok=True)
    
    # پردازش هر تصویر با نوار پیشرفت
    for img_file in tqdm(image_files, desc="پردازش تصاویر"):
        base_name = os.path.splitext(img_file)[0]
        image_path = os.path.join(images_dir, img_file)
        annotation_path = os.path.join(annotations_dir, f"{base_name}.xml")
        
        if os.path.exists(annotation_path):
            extract_plates(image_path, annotation_path, output_dir)
        else:
            print(f"فایل آنوتیشن یافت نشد: {annotation_path}")

if __name__ == "__main__":
    # مسیرهای ورودی و خروجی
    base_dir = "Plates2"
    output_dir = "output_phase1"
    
    # اجرای پردازش
    process_dataset(base_dir, output_dir)
    
    print(f"پردازش فاز اول با موفقیت به پایان رسید. پلاک‌های استخراج شده در پوشه '{output_dir}' ذخیره شدند.")

پردازش تصاویر: 100%|██████████| 217/217 [00:00<00:00, 301.14it/s]

پردازش فاز اول با موفقیت به پایان رسید. پلاک‌های استخراج شده در پوشه 'output_phase1' ذخیره شدند.


# فاز 2

### درست کردن کجی و سیاه سفید کردن

In [18]:
import os
import cv2
import numpy as np

input_folder = 'output_phase1'
output_folder = 'output_phase2'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def angle_from_min_area_rect(thresh):
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None
    largest = max(contours, key=cv2.contourArea)
    rect = cv2.minAreaRect(largest)
    angle = rect[-1]
    if angle < -45:
        angle += 90
    elif angle > 45:
        angle -= 90
    return angle

def angle_from_hough(gray):
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=50, minLineLength=30, maxLineGap=10)
    if lines is None:
        return None
    angles = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
        if -45 < angle < 45:
            angles.append(angle)
    if not angles:
        return None
    return np.median(angles)

def correct_skew_combined(gray_image, angle_threshold=3):
    # Preprocess
    _, thresh = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    thresh = 255 - thresh

    # Step 1: try minAreaRect
    angle = angle_from_min_area_rect(thresh)

    # Step 2: fallback to HoughLines if angle too small or None
    if angle is None or abs(angle) < angle_threshold:
        angle = angle_from_hough(gray_image)

    # اگر همچنان زاویه معنادار نبود، تصویر را برگردان
    if angle is None or abs(angle) < angle_threshold:
        return gray_image

    # Rotate
    (h, w) = gray_image.shape
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(gray_image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

# پردازش همه تصاویر
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        path = os.path.join(input_folder, filename)
        image = cv2.imread(path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        corrected = correct_skew_combined(gray)

        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, corrected)

print("✅ اصلاح زاویه با ترکیب minAreaRect + Hough انجام شد.")


✅ اصلاح زاویه با ترکیب minAreaRect + Hough انجام شد.


### در اوردن کاراکتر های هر عکس

In [45]:
import cv2
import os
import numpy as np
from tqdm import tqdm

def extract_characters(image_path, output_dir, img_idx):
    """استخراج دقیق کاراکترها از تصویر اصلی بدون تغییر"""
    image = cv2.imread(image_path)
    if image is None:
        return

    # تبدیل به خاکستری برای یافتن کانتورها
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # یافتن کانتورها با حداقل پردازش
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # فیلتر کانتورهای نامناسب
    chars = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = w / float(h)
        area = cv2.contourArea(cnt)
        
        # شرایط اصلی برای تشخیص کاراکتر
        if (20 < w < 150 and 40 < h < 200 and 
            0.15 < aspect_ratio < 1.5 and 
            area > 200):
            chars.append((x, y, w, h))

    # اگر کاراکتر کافی نبود، تقسیم مساوی
    if len(chars) < 8:
        h, w = gray.shape
        char_width = w // 8
        chars = [(i*char_width, 0, char_width, h) for i in range(8)]

    # مرتب‌سازی و ذخیره
    chars = sorted(chars[:8], key=lambda c: c[0])
    
    for i, (x, y, w, h) in enumerate(chars, start=1):
        char_img = image[y:y+h, x:x+w]  # استفاده از تصویر اصلی بدون تغییر
        output_path = os.path.join(output_dir, f"{img_idx}.{i}.png")
        cv2.imwrite(output_path, char_img)

def process_phase21(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    for idx, img_file in enumerate(tqdm(image_files, desc="پردازش تصاویر"), start=1):
        input_path = os.path.join(input_dir, img_file)
        extract_characters(input_path, output_dir, idx)

if __name__ == "__main__":
    input_dir = "output_phase2"
    output_dir = "output_phase21"
    process_phase21(input_dir, output_dir)
    print(f"کاراکترها با دقت بالا در '{output_dir}' ذخیره شدند.")

پردازش تصاویر: 100%|██████████| 217/217 [00:00<00:00, 474.63it/s]

کاراکترها با دقت بالا در 'output_phase21' ذخیره شدند.


In [46]:
import cv2
import os
import numpy as np
from tqdm import tqdm

def extract_characters(image_path, output_dir, img_idx):
    """استخراج دقیق کاراکترها از تصویر اصلی با ترتیب خاص"""
    image = cv2.imread(image_path)
    if image is None:
        return

    # تبدیل به خاکستری برای یافتن کانتورها
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # یافتن کانتورها با حداقل پردازش
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # فیلتر کانتورهای نامناسب
    chars = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = w / float(h)
        area = cv2.contourArea(cnt)
        
        # شرایط اصلی برای تشخیص کاراکتر
        if (20 < w < 150 and 40 < h < 200 and 
            0.15 < aspect_ratio < 1.5 and 
            area > 200):
            chars.append((x, y, w, h))

    # اگر کاراکتر کافی نبود، تقسیم مساوی
    if len(chars) < 8:
        h, w = gray.shape
        char_width = w // 8
        chars = [(i*char_width, 0, char_width, h) for i in range(8)]

    # مرتب‌سازی بر اساس موقعیت x و محدود کردن به 8 کاراکتر
    chars = sorted(chars[:8], key=lambda c: c[0])
    
    # ذخیره کاراکترها با نام‌گذاری مناسب برای ترتیب مورد نظر
    for i, (x, y, w, h) in enumerate(chars, start=1):
        char_img = image[y:y+h, x:x+w]
        
        # تعیین نوع کاراکتر بر اساس موقعیت آن
        if i == 1 or i == 2 or i >= 4:  # کاراکترهای عددی
            output_path = os.path.join(output_dir, f"{img_idx}.{i}.number.png")
        elif i == 3:  # کاراکتر حرفی
            output_path = os.path.join(output_dir, f"{img_idx}.{i}.letter.png")
        
        cv2.imwrite(output_path, char_img)

def process_phase21(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    for idx, img_file in enumerate(tqdm(image_files, desc="پردازش تصاویر"), start=1):
        input_path = os.path.join(input_dir, img_file)
        extract_characters(input_path, output_dir, idx)

if __name__ == "__main__":
    input_dir = "output_phase2"
    output_dir = "output_phase21"
    process_phase21(input_dir, output_dir)
    print(f"کاراکترها با ترتیب صحیح در '{output_dir}' ذخیره شدند.")

پردازش تصاویر: 100%|██████████| 217/217 [00:00<00:00, 479.40it/s]

کاراکترها با ترتیب صحیح در 'output_phase21' ذخیره شدند.
